In [1]:
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import skimage
from datetime import datetime
from IPython.display import clear_output
import sys
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import geopandas
import os
import random
from datetime import datetime
import time
import pytz
import shutil
import wget
from suntime import Sun
from datetime import timedelta
from datetime import date


In [17]:
smoke_dir = '/scratch1/RDARCH/rda-ghpcs/Rey.Koki/smoke/'

def get_smoke_fn_url(dt):
    tt = dt.timetuple()
    month = str(tt.tm_mon).zfill(2)
    day = str(tt.tm_mday).zfill(2)
    yr = str(tt.tm_year)
    fn = 'hms_smoke{}{}{}.zip'.format(yr, month, day)
    url = 'https://satepsanone.nesdis.noaa.gov/pub/FIRE/web/HMS/Smoke_Polygons/Shapefile/{}/{}/{}'.format(yr, month, fn)
    return fn, url

def get_sunrise_sunset(dt, lat, lon):
    east = Sun(lat, lon)
    west = Sun(lat, lon)
    sunset = east.get_sunset_time(dt)
    sunrise = west.get_sunrise_time(dt)
    if sunrise > sunset:
        sunset = west.get_sunset_time(dt + timedelta(days=1))
    print('\nsunrise: ', sunrise)
    print('sunset: ', sunset)
    return sunrise, sunset

def get_idx(smoke_shape, dt, idx_oi):
    use_idx = []
    if smoke_shape is not None:
        fmt = '%Y%j %H%M'
        smoke = smoke_shape
        start_ends = []
        centers = smoke.centroid
        for idx, row in smoke.iterrows():
            start = pytz.utc.localize(datetime.strptime(smoke.loc[idx]['Start'], fmt))
            end = pytz.utc.localize(datetime.strptime(smoke.loc[idx]['End'], fmt))
            start_ends.append((start, end))
            if idx == int(idx_oi):
                print('\nanalyst timewindow: ', end-start)
                center = centers.loc[idx]
                print(row)
                print(center)
                #print(center.x)
                #print(center.y)
                use_idx.append(idx)
                get_sunrise_sunset(dt, center.y, center.x)
    if use_idx:
        smoke = smoke_shape.loc[use_idx]
    return smoke, use_idx

def get_smoke(dt, idx_oi):
    fn, url = get_smoke_fn_url(dt)
    print('DOWNLOADING SMOKE:')
    print(fn)
    smoke_shape_fn = smoke_dir + fn
    print(smoke_shape_fn)
    if os.path.exists(smoke_dir+fn):
        print("{} already exists".format(fn))
        smoke = geopandas.read_file(smoke_shape_fn)
        smoke, use_idx = get_idx(smoke, dt, idx_oi)
        return smoke, use_idx
    else:
        try:
            filename = wget.download(url, out=smoke_dir)
            shutil.unpack_archive(filename, smoke_dir)
            smoke = geopandas.read_file(smoke_shape_fn)

            smoke, use_idx = get_idx(smoke, dt)
            return smoke, use_idx
        except:
            print('NO SMOKE DATA FOR THIS DATE')
            return None, []


def get_dn(yr_mo_day):
    fmt = '%Y%M%d'
    dt = datetime.strptime(yr_mo_day, fmt)
    dt = pytz.utc.localize(dt)
    return dt

def get_dt(year, dn):
    fmt = '%Y%j'
    yr_dn = year+dn
    dt = datetime.strptime(yr_dn, fmt)
    dt = pytz.utc.localize(dt)
    return dt

def fn_info(fn):
    fn = fn.split('_')
    yr = fn[1][1:5]
    dn = fn[1][5:8]
    idx_oi = fn[-1].split('.')[0]
    return yr, dn, idx_oi
    
def get_command(fn):
    fn = fn.split('_')
    yr = fn[1][1:5]
    dn = fn[1][5:8]
    idx_oi = fn[-1].split('.')[0]
    
    return yr, dn, idx_oi

# analysts can only label data that is taken during the daytime, we want to filter for geos data that was within the timeframe the analysts are looking at
def check_annotation(fn):
    yr, dn, idx_oi = fn_info(fn)
    dt = get_dt(yr,dn)
    smoke, use_idx = get_smoke(dt,idx_oi)
    print(smoke)
    print('sbatch --export=YEAR={},DN={},IDX={} run.script'.format(yr, dn, idx_oi))
    return smoke

#check_annotation('G17_s20222891610321_e20222891619388_89.tif')
#check_annotation('G17_s20220021230321_18.53_-72.32_19.tif')
check_annotation('G16_s20220042300207_32.91_-93.18_194.tif')

DOWNLOADING SMOKE:
hms_smoke20220104.zip
/scratch1/RDARCH/rda-ghpcs/Rey.Koki/smoke/hms_smoke20220104.zip
hms_smoke20220104.zip already exists

analyst timewindow:  4:00:00
Satellite                                            GOES-EAST
Start                                             2022004 2100
End                                               2022005 0100
Density                                                  Heavy
geometry     POLYGON ((-93.001342 31.154252, -92.94803 31.2...
Name: 194, dtype: object
POINT (-92.70907494318107 31.663490816080337)

sunrise:  2022-01-04 13:11:24+00:00
sunset:  2022-01-04 23:20:24+00:00
     Satellite         Start           End Density  \
194  GOES-EAST  2022004 2100  2022005 0100   Heavy   

                                              geometry  
194  POLYGON ((-93.00134 31.15425, -92.94803 31.267...  
sbatch --export=YEAR=2022,DN=004,IDX=194 run.script


/tmp/ipykernel_1372887/879118912.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centers = smoke.centroid


,Satellite,Start,End,Density,geometry
194,GOES-EAST,2022004 2100,2022005 0100,Heavy,"POLYGON ((-93.00134 31.15425, -92.94803 31.267..."
